<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# $H_{\rm Orb, NS}$, up to and including third post-Newtonian order

## This notebook constructs the orbital, nonspinning Hamiltonian up to third post-Newtonian order, as summarized in [Buonanno, Chen, and Damour (2006)](https://arxiv.org/abs/gr-qc/0508067) (see references therein for sources)

**Notebook Status:** <font color='orange'><b> Self-validated </b></font>

**Validation Notes:** This tutorial notebook has been confirmed to be self-consistent with its corresponding NRPy+ module, as documented [below](#code_validation). **Additional validation tests may have been performed, but are as yet, undocumented. (TODO)**

## Author: Zach Etienne

### This notebook exists as the following Python module:
1. [PN_Hamiltonian_NS.py](../../edit/NRPyPN/PN_Hamiltonian_NS.py)

### This notebook & corresponding Python module depend on the following NRPy+/NRPyPN Python modules:
1. [outputC.py](../../edit/outputC.py): [**documentation+tutorial**](../Tutorial-Coutput__Parameter_Interface.ipynb)
1. [indexedexp.py](../../edit/indexedexp.py): [**documentation+tutorial**](../Tutorial-Indexed_Expressions.ipynb)
1. [NRPyPN_shortcuts.py](../../edit/NRPyPN/NRPyPN_shortcuts.py): [**documentation**](NRPyPN_shortcuts.ipynb)

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

1. Part 1: [$H_{\rm Orb,\ NS,\ Newt}+H_{\rm Orb,\ NS,\ 1PN}+H_{\rm Orb,\ NS,\ 2PN}$](#uptotwopn): Contributions up to and including second post-Newtonian order
1. Part 2: [$H_{\rm Orb,\ NS,\ 3PN}$](#threepn): Third post-Newtonian contribution
1. Part 3: [Validation against corresponding Python module](#code_validation)
1. Part 4: [LaTeX PDF output](#latex_pdf_output): $\LaTeX$ PDF Output

<a id='uptotwopn'></a>

# Part 1: $H_{\rm Orb,\ NS,\ Newt}+H_{\rm Orb,\ NS,\ 1PN}+H_{\rm Orb,\ NS,\ 2PN}$ \[Back to [top](#toc)\]
$$\label{uptotwopn}$$ 

As detailed in [Buonanno, Chen, and Damour (2006)](https://arxiv.org/abs/gr-qc/0508067) (henceforth BCD2006),
this model assumes two point masses of mass $m_1$ and $m_2$ with corresponding momentum vectors $\mathbf{P}_1$ and $\mathbf{P}_2$, and displacement vectors $\mathbf{X}_1$ and $\mathbf{X}_2$ with respect to the center of mass.

Following [BCD2006](https://arxiv.org/abs/gr-qc/0508067), we define the following quantities
\begin{align}
\mu &= m_1 m_2 / (m_1+m_2)^2\\
\mathbf{p} &= \mathbf{P}_1/\mu = -\mathbf{P}_2/\mu\\
\mathbf{q} &= (\mathbf{X}_1-\mathbf{X}_2)/M\\
q &= |\mathbf{q}|\\
\mathbf{n} &= \frac{\mathbf{q}}{q}
\end{align}

Then the Hamiltonian up to and including second PN order is given by (to reduce possibility of copying error, these equations are taken directly from Eqs 2.2-4 of the LaTeX source code of [BCD2006](https://arxiv.org/abs/gr-qc/0508067), and only mildly formatted to (1) improve presentation in Jupyter notebooks and (2) to ensure some degree of consistency in notation across different terms in other Hamiltonian notebooks):
\begin{align}
H_{\rm Newt}\left({\bf q},{\bf p}\right) &= \mu \left[\frac{{\bf p}^2}{2} -
\frac{1}{q}\right]\,, \\
H_{\rm 1PN}\left({\bf q},{\bf p}\right) &= \mu\left[\frac{1}{8}(3\eta-1)({\bf p}^2)^2
- \frac{1}{2}\left[(3+\eta){\bf p}^2+\eta({\bf n}\cdot{\bf p})^2\right]\frac{1}{q} + \frac{1}{2q^2}\right]\,,\\
H_{\rm 2PN}\left({\bf q},{\bf p}\right)
&= \mu\left[\frac{1}{16}\left(1-5\eta+5\eta^2\right)({\bf p}^2)^3
+ \frac{1}{8} \left[
\left(5-20\eta-3\eta^2\right)({\bf p}^2)^2-2\eta^2({\bf n}\cdot{\bf p})^2{\bf p}^2-3\eta^2({\bf n}\cdot{\bf p})^4 \right]\frac{1}{q}\right.
\\
&\quad\quad\quad \left.+ \frac{1}{2} \left[(5+8\eta){\bf p}^2+3\eta({\bf n}\cdot{\bf p})^2\right]\frac{1}{q^2}
- \frac{1}{4}(1+3\eta)\frac{1}{q^3}\right]\,,
\end{align}

In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys                    # Standard Python modules for multiplatform OS-level functions
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)
import sympy as sp               # SymPy: The Python computer algebra package upon which NRPy+ depends
from outputC import *            # NRPy+: Core C code output module
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
from NRPyPN_shortcuts import *   # NRPyPN: shortcuts for e.g., vector operations

def f_H_Newt__H_NS_1PN__H_NS_2PN(m1,m2, pU, nU, q):
    mu = m1*m2/(m1+m2)
    PU = ixp.zerorank1()
    for i in range(3):
        PU[i] = pU[i]/mu
    eta = m1*m2/(m1+m2)**2
    P_dot_P = dot(PU,PU)
    n_dot_P = dot(nU,PU)

    # H_{\rm Newt} = \frac{p^i p^i}{2} - \frac{1}{q}
    global H_Newt,H_NS_1PN,H_NS_2PN
    H_Newt    = mu*(div(1,2)*P_dot_P - 1/q)
    H_NS_1PN  = mu*(div(1,8)*(3*eta-1)*P_dot_P**2 - \
                 div(1,2)*((3+eta)*P_dot_P + eta*n_dot_P**2)/q + 1/(2*q**2))
    H_NS_2PN  = mu*(div(1,16)*(1 - 5*eta + 5*eta**2)*P_dot_P**3 +
                 div(1,8)*((5 - 20*eta - 3*eta**2)*P_dot_P**2 
                           - 2*eta**2*n_dot_P**2*P_dot_P - 3*eta**2*n_dot_P**4)/q +
                 div(1,2)*((5 + 8*eta)*P_dot_P + 3*eta*n_dot_P**2)/q**2 -
                 div(1,4)*(1 + 3*eta)/q**3)

<a id='threepn'></a>

# Part 2: $H_{\rm Orb,\ NS,\ 3PN}$ Third post-Newtonian contribution \[Back to [top](#toc)\]
$$\label{threepn}$$ 

To reduce possibility of copying error, equations are taken directly from the LaTeX source code of Eqs 2.2-4 in [BCD2006](https://arxiv.org/abs/gr-qc/0508067), and only mildly formatted to (1) improve presentation in Jupyter notebooks and (2) to ensure some degree of consistency in notation across different terms in other Hamiltonian notebooks:

\begin{align}
H_{\rm 3PN}\left({\bf q},{\bf p}\right)
&= \frac{1}{128}\left(-5+35\eta-70\eta^2+35\eta^3\right)({\bf p}^2)^4
 \\
&\quad + \frac{1}{16}\left[
\left(-7+42\eta-53\eta^2-5\eta^3\right)({\bf p}^2)^3
+ (2-3\eta)\eta^2({\bf n}\cdot{\bf p})^2({\bf p}^2)^2
+ 3(1-\eta)\eta^2({\bf n}\cdot{\bf p})^4{\bf p}^2 - 5\eta^3({\bf n}\cdot{\bf p})^6
\right]\frac{1}{q}
 \\
&\quad +\left[ \frac{1}{16}\left(-27+136\eta+109\eta^2\right)({\bf p}^2)^2
+ \frac{1}{16}(17+30\eta)\eta({\bf n}\cdot{\bf p})^2{\bf p}^2 + \frac{1}{12}(5+43\eta)\eta({\bf n}\cdot{\bf p})^4
\right]\frac{1}{q^2} \\
&\quad +\left\{ \left[ -\frac{25}{8} + \left(\frac{1}{64}\pi^2-\frac{335}{48}\right)\eta 
- \frac{23}{8}\eta^2 \right]{\bf p}^2
+ \left(-\frac{85}{16}-\frac{3}{64}\pi^2-\frac{7}{4}\eta\right)\eta({\bf n}\cdot{\bf p})^2 
\right\}\frac{1}{q^3}
\\
&\quad + \left[ \frac{1}{8} + \left(\frac{109}{12}-\frac{21}{32}\pi^2\right)\eta 
\right]\frac{1}{q^4}\,,
\end{align}

In [2]:
def f_H_NS_3PN(m1,m2, pU, nU, q):
    mu = m1*m2/(m1+m2)
    PU = ixp.zerorank1()
    for i in range(3):
        PU[i] = pU[i]/mu
    eta = m1*m2/(m1+m2)**2
    P_dot_P = dot(PU,PU)
    n_dot_P = dot(nU,PU)

    global H_NS_3PN
    # The following is simply by-hand search/replaced from the above LaTeX to minimize error
    H_NS_3PN = \
    mu*( div(1,128)*(-5+35*eta-70*eta**2+35*eta**3)*P_dot_P**4 +
         div(1,16)* ( (-7+42*eta-53*eta**2-5*eta**3)*P_dot_P**3
                     +(2-3*eta)*eta**2*n_dot_P**2*P_dot_P**2 +
                     +3*(1-eta)*eta**2*n_dot_P**4*P_dot_P - 5*eta**3*n_dot_P**6 )/(q) +
 (  div(1,16)*(-27+136*eta+109*eta**2)*P_dot_P**2
 + div(1,16)*(17+30*eta)*eta*n_dot_P**2*P_dot_P + div(1,12)*(5+43*eta)*eta*n_dot_P**4)/(q**2) +
 ( ( -div(25,8) + (div(1,64)*sp.pi**2-div(335,48))*eta 
 - div(23,8)*eta**2 )*P_dot_P
 + (-div(85,16)-div(3,64)*sp.pi**2-div(7,4)*eta)*eta*n_dot_P**2)/(q**3) +
 ( div(1,8) + (div(109,12)-div(21,32)*sp.pi**2)*eta)/(q**4) )

<a id='code_validation'></a>

# Part 3: Validation against corresponding Python module \[Back to [top](#toc)\]
$$\label{code_validation}$$ 

As a code validation check, we verify agreement between the SymPy expressions generated in this notebook, and the corresponding Python module.

In [3]:
from outputC import *   # NRPy+: Core C code output module

f_H_Newt__H_NS_1PN__H_NS_2PN(m1,m2, pU, nU, q)
f_H_NS_3PN(m1,m2, pU, nU, q)

import PN_Hamiltonian_NS as HNS
HNS.f_H_Newt__H_NS_1PN__H_NS_2PN(m1,m2, pU, nU, q)
HNS.f_H_NS_3PN(m1,m2, pU, nU, q)

def error(varname):
    print("ERROR: When comparing Python module & notebook, "+varname+" was found not to match.")
    sys.exit(1)

if sp.simplify(H_Newt   - HNS.H_Newt)   != 0: error("H_Newt")
if sp.simplify(H_NS_1PN - HNS.H_NS_1PN) != 0: error("H_NS_1PN")
if sp.simplify(H_NS_2PN - HNS.H_NS_2PN) != 0: error("H_NS_2PN")
print("ALL TESTS PASS")

ALL TESTS PASS


<a id='latex_pdf_output'></a>

# Step 4: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[PN-Hamiltonian-Nonspinning.pdf](PN-Hamiltonian-Nonspinning.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [4]:
import os,sys                    # Standard Python modules for multiplatform OS-level functions
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("PN-Hamiltonian-Nonspinning",location_of_template_file=os.path.join(".."))

Created PN-Hamiltonian-Nonspinning.tex, and compiled LaTeX file to PDF file PN-Hamiltonian-Nonspinning.pdf
